In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [ ]:
(ds_train,ds_test),ds_info=tfds.load(
    'mnist',
    split=['train','test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
def preprocessing(image,label):
  return tf.cast(image,tf.float32)/255.0,label

In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
BATCH_SIZE=32
ds_train=ds_train.map(preprocessing,num_parallel_calls=AUTOTUNE)
ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE)
ds_test=ds_test.map(preprocessing)
ds_test=ds_test.batch(BATCH_SIZE)
ds_test=ds_test.prefetch(AUTOTUNE)

In [ ]:
model=keras.Sequential(
    [keras.Input(shape=(28,28,1)),
     keras.layers.Conv2D(32,3,activation='relu'),
     keras.layers.MaxPooling2D(),
     keras.layers.Flatten(),
     keras.layers.Dense(10,activation='softmax')]
)


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train,validation_data=ds_test,epochs=10,verbose=2)

Epoch 1/10
1875/1875 - 24s - 13ms/step - accuracy: 0.9005 - loss: 0.3720 - val_accuracy: 0.9451 - val_loss: 0.1870
Epoch 2/10
1875/1875 - 11s - 6ms/step - accuracy: 0.9562 - loss: 0.1552 - val_accuracy: 0.9665 - val_loss: 0.1201
Epoch 3/10
1875/1875 - 10s - 5ms/step - accuracy: 0.9706 - loss: 0.1083 - val_accuracy: 0.9735 - val_loss: 0.0929
Epoch 4/10
1875/1875 - 10s - 5ms/step - accuracy: 0.9766 - loss: 0.0858 - val_accuracy: 0.9761 - val_loss: 0.0788
Epoch 5/10
1875/1875 - 7s - 4ms/step - accuracy: 0.9800 - loss: 0.0724 - val_accuracy: 0.9779 - val_loss: 0.0707
Epoch 6/10
1875/1875 - 10s - 5ms/step - accuracy: 0.9823 - loss: 0.0635 - val_accuracy: 0.9789 - val_loss: 0.0658
Epoch 7/10
1875/1875 - 10s - 5ms/step - accuracy: 0.9839 - loss: 0.0570 - val_accuracy: 0.9794 - val_loss: 0.0628
Epoch 8/10
1875/1875 - 6s - 3ms/step - accuracy: 0.9855 - loss: 0.0519 - val_accuracy: 0.9798 - val_loss: 0.0607
Epoch 9/10
1875/1875 - 10s - 5ms/step - accuracy: 0.9869 - loss: 0.0477 - val_accuracy: 0

In [ ]:
epochs=10
loss_fn=keras.losses.SparseCategoricalCrossentropy()
optimizer=keras.optimizers.Adam(3e-4)
acc_metric=keras.metrics.SparseCategoricalAccuracy()

In [ ]:
for epoch in range(epochs):
  print(f'\nTraining started for epoch {epoch}')
  for batch_idx,(x_batch,y_batch) in enumerate(ds_train):
    with tf.GradientTape() as tape:
      y_pred=model(x_batch,training=True)
      loss=loss_fn(y_batch,y_pred)
    gradients=tape.gradient(loss,model.trainable_weights)
    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    acc_metric.update_state(y_batch,y_pred)
  train_accuracy=acc_metric.result()
  print(f'Accuracy over epoch {train_accuracy}')
  acc_metric.reset_state()



Training started for epoch 0
Accuracy over epoch 0.9886666536331177

Training started for epoch 1
Accuracy over epoch 0.9896833300590515

Training started for epoch 2
Accuracy over epoch 0.9906666874885559

Training started for epoch 3
Accuracy over epoch 0.9912499785423279

Training started for epoch 4


In [ ]:
#Test Loop
for batch_idx,(x_batch,y_batch) in enumerate(ds_test):
  y_pred=model(x_batch,training=False)
  acc_metric.update_state(y_batch,y_pred)
test_accuracy=acc_metric.result()
print(f'Accuracy over test set {test_accuracy}')
acc_metric.reset_state()

Accuracy over test set 0.9835000038146973
